In [2]:
# ETL output_steam_games.json

# Importación de librerías
import pandas as pd
import datetime
import pyarrow as pa
import pyarrow.parquet as pq

# Obtener datos del archivo json
games = pd.read_json('../data_sources/json/output_steam_games.json', lines=True)

# Eliminar las filas nulas
games = games.dropna(how="all").reset_index(drop=True)

# Eliminar las columnas innecesarias
games = games.drop(columns=['publisher', 'url', 'app_name', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'developer'])

# Buscar y eliminar filas duplicadas
games['genres'] = games['genres'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
duplicates = games[games.duplicated(keep='first', subset=['id'])]
duplicates = duplicates.index.tolist()
games = games.drop(duplicates).reset_index(drop=True)

# Crear una nueva columna con el año de lanzamiento y eliminar la columna de fecha de lanzamiento
games['release_year'] = games['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
games = games.drop(columns=['release_date'])

# Elimina las filas donde el campo title sea nulo
games = games.dropna(subset=['title'])

# Valida que el año de lanzamiento no sea mayor al año actual
current_year = datetime.datetime.now().year
games['release_year'] = games['release_year'].apply(lambda x: x if x <= current_year else 0)

# Convierte el DataFrame de Pandas a una tabla Arrow
table = pa.Table.from_pandas(games)

# Especifica la ruta del archivo Parquet
file = '../data_sources/parquet/steam_games.parquet'

# Escribe la tabla Arrow en un archivo Parquet
pq.write_table(table, file, compression='gzip')